# Grand Circus Final Project
### Car Crash and Safety Data Comparisons/Evaluations

This project aims to compare safety ratings from crash tests to actual data of fatal crashes. The use of fatal crash data is better suited for hard crashes where occupant life is and was in danger, providing more relevant data entries compared to fender benders or other minimal 'traffic incidents'. This analysis could be useful for car buyers, car manufactureres, government testers, and insurance companies.

## Extraction
To start the ETA process, data but be extracted and placed into usable structures. To do this, we will be importing the data from the api(s) and any other flat file sources.

In [161]:
import pandas as pd
import requests
import json

# Begin pulling make names and ID's for internal use
# Definitions endpoint query
make_url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/definitions/GetVariableAttributes?variable=make&caseYear=2010&format=json"

# Base URL for NHTSA API
#base_url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/crashes/"
#params = "GetCrashesByVehicle?make=6&model=41&modelyear=2003&bodyType=4&fromCaseYear=2010&toCaseYear=2015&state=21&format=json"

# Get response
response = requests.get(make_url)
# Turn response into json
data = response.json()

In [163]:
#Drill down json to list of dictionary
results = data['Results'][0]

In [159]:
# split data into lists
id_list = []
name_list = []
for entry in results:
    id_list.append(entry['ID'])
    name_list.append(entry['TEXT'])

# Make columns dictionary based on lists
data = {'ID': id_list, 'Name': name_list}

# Create df using dictionary
manufacturer_df = pd.DataFrame(data)

# Sort by Id instead of name
manufacturer_df = manufacturer_df.sort_values(by=['ID'])
manufacturer_df.head()

,ID,Name
3,1,American Motors
19,10,Eagle
22,12,Ford
44,13,Lincoln
49,14,Mercury


## Transformation
Now that we have usable, workable data, we can begin cleaning and organizing.

In [ ]:
# Transformation code

# Drop any unneeded columns/rows
    # duplicates
    # nulls
    # outliers

# Merge/Join Data into one dataframe



## Analysis
With curated data, analysis can begin to check for trends, patters, and correlations.

In [ ]:
# Analysis stuff

# list stats
    # most common car with fatalities
    # that kinda stuff, idk

# can probably save visualizing for PowerBI

# more stuff I'm probably forgetting